In [ ]:
import os
import json
import pandas as pd

input_directory = 'traficdatatest1'
output_file = 'timediff10.txt'
timediff_lines = []

# Iterate over each file in the directory
for filename in os.listdir(input_directory):
    if filename.startswith('traffic.txt'):
        file_path = os.path.join(input_directory, filename)
        with open(file_path, 'r') as file:
            for line in file:
                try:
                    record = json.loads(line)
                    if "save_dto" in record:
                        dto = pd.to_datetime(record["dto"])
                        save_dto = pd.to_datetime(record["save_dto"])
                        time_diff = (save_dto - dto).total_seconds()
                        if time_diff > 10:
                            timediff_lines.append(line.strip())
                except json.JSONDecodeError:
                    continue

# Save the lines to the output file
with open(output_file, 'w') as outfile:
    outfile.write('\n'.join(timediff_lines))


In [10]:
import paramiko
import json
import os

# SFTP connection details
hostname = 'files.clarksonmsda.org'
port = 2022
username = 'kadaliu@clarkson.edu'
password = 'Act.324fgdg'
remote_prefix = '/srv/sftpgo/data/users/kadaliu@clarkson.edu/'

# Local directory to save images
local_directory = '/Users/tejeshchintpalli/Downloads/capstone/timediff10'

# Path to the text file
text_file_path = '/Users/tejeshchintpalli/Downloads/capstone/timediff10sample.txt'

# Create local directory if it doesn't exist
os.makedirs(local_directory, exist_ok=True)

# Read JSON entries from the text file
with open(text_file_path, 'r') as file:
    json_entries = [json.loads(line) for line in file]

# SFTP connection setup
print("Connecting to SFTP server...")
try:
    transport = paramiko.Transport((hostname, port))
    transport.connect(username=username, password=password)
    sftp = paramiko.SFTPClient.from_transport(transport)
    print("Connected to SFTP server.")
except Exception as e:
    print(f"Failed to connect to SFTP server: {e}")
    exit(1)

# Download each image
for entry in json_entries:
    remote_file_path = remote_prefix + entry['full_img']
    local_file_path = os.path.join(local_directory, os.path.basename(entry['full_img']))
    print(f"Downloading {remote_file_path} to {local_file_path}...")
    try:
        sftp.get(remote_file_path, local_file_path)
        print(f"Downloaded: {local_file_path}")
    except Exception as e:
        print(f"Failed to download {remote_file_path}: {e}")

# Close SFTP connection
sftp.close()
transport.close()
print("SFTP connection closed.")


Connecting to SFTP server...
Connected to SFTP server.
Downloaded: /Users/tejeshchintpalli/Downloads/capstone/timediff10/073402_15659.jpg
Downloaded: /Users/tejeshchintpalli/Downloads/capstone/timediff10/074302_15683.jpg
Downloaded: /Users/tejeshchintpalli/Downloads/capstone/timediff10/075301_15735.jpg
Downloaded: /Users/tejeshchintpalli/Downloads/capstone/timediff10/081404_15818.jpg
Downloaded: /Users/tejeshchintpalli/Downloads/capstone/timediff10/083702_15910.jpg
Downloaded: /Users/tejeshchintpalli/Downloads/capstone/timediff10/083901_15919.jpg
Downloaded: /Users/tejeshchintpalli/Downloads/capstone/timediff10/091803_16133.jpg
Downloaded: /Users/tejeshchintpalli/Downloads/capstone/timediff10/092202_16169.jpg
Downloaded: /Users/tejeshchintpalli/Downloads/capstone/timediff10/093508_16279.jpg
Downloaded: /Users/tejeshchintpalli/Downloads/capstone/timediff10/093608_16287.jpg
SFTP connection closed.


In [12]:
pip install pillow



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
import json
import os
from PIL import Image, ImageDraw, ImageFont

# Path to the text file
text_file_path = '/Users/tejeshchintpalli/Downloads/capstone/timediff10sample.txt'

# Directory containing the images
image_directory = '/Users/tejeshchintpalli/Downloads/capstone/timediff10'
# Directory to save the modified images
save_directory = '/Users/tejeshchintpalli/Downloads/capstone/timediff10save'

# Create save directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Load JSON entries from the text file
with open(text_file_path, 'r') as file:
    json_entries = [json.loads(line) for line in file]

# Map full_img paths to their save_dto values
save_dto_map = {os.path.basename(entry['full_img']): entry['save_dto'] for entry in json_entries}

# Function to add save_dto text to an image
def add_text_to_image(image_path, text, save_path):
    try:
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)
        
        # Use a built-in truetype font
        font = ImageFont.load_default()
        # Or specify a truetype font
        # font = ImageFont.truetype("arial.ttf", 40)

        # Calculate text size and position using textbbox
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        width, height = image.size
        x = width - text_width - 20
        y = height - text_height - 20

        # Draw a rectangle as background for the text
        draw.rectangle(((x - 5, y - 5), (x + text_width + 5, y + text_height + 5)), fill="black")

        # Add text to image
        draw.text((x, y), text, font=font, fill="white")

        # Save the image
        image.save(save_path)
        print(f"Added text '{text}' to {save_path}")
    except Exception as e:
        print(f"Failed to add text to {image_path}: {e}")

# Process each image in the directory
for image_filename in os.listdir(image_directory):
    if image_filename in save_dto_map:
        image_path = os.path.join(image_directory, image_filename)
        save_path = os.path.join(save_directory, image_filename)
        add_text_to_image(image_path, save_dto_map[image_filename], save_path)

print("Processing complete.")


Processing complete.


In [22]:
import json
import os
from PIL import Image, ImageDraw, ImageFont

# Path to the text file
text_file_path = '/Users/tejeshchintpalli/Downloads/capstone/timediff10sample.txt'

# Directory containing the images
image_directory = '/Users/tejeshchintpalli/Downloads/capstone/timediff10'
# Directory to save the modified images
save_directory = '/Users/tejeshchintpalli/Downloads/capstone/timediff10save'

# Create save directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Load JSON entries from the text file
with open(text_file_path, 'r') as file:
    json_entries = [json.loads(line) for line in file]

# Map full_img paths to their save_dto values
save_dto_map = {os.path.basename(entry['full_img']): entry['save_dto'] for entry in json_entries}

# Function to add save_dto text to an image
def add_text_to_image(image_path, text, save_path):
    try:
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)

        # Use a default bitmap font and set a larger size
        font = ImageFont.load_default()
        text_scale = 3  # Scale factor for the default font

        # Calculate text size and position
        text_width, text_height = draw.textsize(text, font=font)
        text_width *= text_scale
        text_height *= text_scale
        width, height = image.size
        x = width - text_width - 20
        y = height - text_height - 20

        # Draw a rectangle as background for the text
        draw.rectangle(((x - 5, y - 5), (x + text_width + 5, y + text_height + 5)), fill="black")

        # Add text to image
        for i in range(text_scale):
            for j in range(text_scale):
                draw.text((x + i, y + j), text, font=font, fill="white")

        # Save the image
        image.save(save_path)
        print(f"Added text '{text}' to {save_path}")
    except Exception as e:
        print(f"Failed to add text to {image_path}: {e}")

# Process each image in the directory
for image_filename in os.listdir(image_directory):
    if image_filename in save_dto_map:
        image_path = os.path.join(image_directory, image_filename)
        save_path = os.path.join(save_directory, image_filename)
        add_text_to_image(image_path, save_dto_map[image_filename], save_path)

print("Processing complete.")


Failed to add text to /Users/tejeshchintpalli/Downloads/capstone/timediff10/093508_16279.jpg: 'ImageDraw' object has no attribute 'textsize'
Failed to add text to /Users/tejeshchintpalli/Downloads/capstone/timediff10/083901_15919.jpg: 'ImageDraw' object has no attribute 'textsize'
Failed to add text to /Users/tejeshchintpalli/Downloads/capstone/timediff10/075301_15735.jpg: 'ImageDraw' object has no attribute 'textsize'
Failed to add text to /Users/tejeshchintpalli/Downloads/capstone/timediff10/093608_16287.jpg: 'ImageDraw' object has no attribute 'textsize'
Failed to add text to /Users/tejeshchintpalli/Downloads/capstone/timediff10/091803_16133.jpg: 'ImageDraw' object has no attribute 'textsize'
Failed to add text to /Users/tejeshchintpalli/Downloads/capstone/timediff10/074302_15683.jpg: 'ImageDraw' object has no attribute 'textsize'
Failed to add text to /Users/tejeshchintpalli/Downloads/capstone/timediff10/083702_15910.jpg: 'ImageDraw' object has no attribute 'textsize'
Failed to add

In [41]:
import json
import os
from PIL import Image, ImageDraw, ImageFont

# Path to the text file
text_file_path = '/Users/tejeshchintpalli/Downloads/capstone/timediff10sample.txt'

# Directory containing the images
image_directory = '/Users/tejeshchintpalli/Downloads/capstone/timediff10'
# Directory to save the modified images
save_directory = '/Users/tejeshchintpalli/Downloads/capstone/timediff10save'

# Create save directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Load JSON entries from the text file
with open(text_file_path, 'r') as file:
    json_entries = [json.loads(line) for line in file]

# Map full_img paths to their save_dto values
save_dto_map = {os.path.basename(entry['full_img']): entry['save_dto'] for entry in json_entries}

# Function to add save_dto text to an image
def add_text_to_image(image_path, text, save_path):
    try:
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)
        
        # Specify a truetype font and size
        font = ImageFont.truetype("arial.ttf", 40)  # Make sure arial.ttf is available or specify another available font

        # Calculate text size and position using textbbox
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        width, height = image.size
        x = width - text_width - 20
        y = height - text_height - 20

        # Draw a rectangle as background for the text
        draw.rectangle(((x - 5, y - 5), (x + text_width + 5, y + text_height + 5)), fill="black")

        # Add text to image
        draw.text((x, y), text, font=font, fill="white")

        # Save the image
        image.save(save_path)
        print(f"Added text '{text}' to {save_path}")
    except Exception as e:
        print(f"Failed to add text to {image_path}: {e}")

# Process each image in the directory
for image_filename in os.listdir(image_directory):
    if image_filename in save_dto_map:
        image_path = os.path.join(image_directory, image_filename)
        save_path = os.path.join(save_directory, image_filename)
        add_text_to_image(image_path, save_dto_map[image_filename], save_path)

print("Processing complete.")


Processing complete.
